In [1]:
import cv2
import numpy as np
from twilio.rest import Client
import pygame
import tkinter as tk
from tkinter import ttk
from threading import Thread
import time

pygame 2.6.1 (SDL 2.28.4, Python 3.11.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
account_sid = 'twilio_account_sid'
auth_token = 'twilio_auth_token'
twilio_phone_number = 'twilio_phone_number'
user_phone_number = 'your_phone_number'

In [3]:
client = Client(account_sid, auth_token)

In [4]:
def send_alert(message):
    """Send SMS alert using Twilio."""
    client.messages.create(
        body=message,
        from_=twilio_phone_number,
        to=user_phone_number
    )
    print("Alert sent to user.")

In [5]:
pygame.init()
SIREN_PATH = r"path to siren.wav"
siren_sound = pygame.mixer.Sound(SIREN_PATH)

In [6]:
print("Loading custom YOLOv4 model...")
net = cv2.dnn.readNet(r'path to handgun_yolov4.weights', r'path to yolov4-obj.cfg')
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

Loading custom YOLOv4 model...


In [7]:
with open(r'path to obj.names', 'r') as f:
    classes = f.read().strip().split('\n')

In [8]:
class WeaponDetectionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Weapon Detection System")
        self.root.geometry("800x500")
        self.root.config(bg="#2E2E2E")

        self.is_running = False
        self.siren_playing = False
        self.video_capture = None
        self.last_alert_time = 0

        header = tk.Label(self.root, text="Weapon Detection System", 
                          font=("Helvetica", 24, "bold"), bg="#3B3B3B", fg="white")
        header.pack(fill=tk.X, pady=(10, 0))

        main_frame = ttk.Frame(self.root, padding=20)
        main_frame.pack(expand=True, fill=tk.BOTH)

        self.status_label = ttk.Label(main_frame, text="Status: Idle", 
                                      font=("Arial", 18), foreground="#FF6347")
        self.status_label.pack(pady=10)

        button_frame = ttk.Frame(main_frame)
        button_frame.pack(pady=20)

        self.start_button = ttk.Button(button_frame, text="Start Detection", 
                                       command=self.start_detection)
        self.start_button.grid(row=0, column=0, padx=10)

        self.stop_button = ttk.Button(button_frame, text="Stop Detection", 
                                      command=self.stop_detection)
        self.stop_button.grid(row=0, column=1, padx=10)

        self.quit_button = ttk.Button(button_frame, text="Quit", 
                                      command=self.quit_application)
        self.quit_button.grid(row=0, column=2, padx=10)

    def start_detection(self):
        """Start the detection thread."""
        if not self.is_running:
            self.is_running = True
            self.video_capture = cv2.VideoCapture(0)
            self.status_label.config(text="Status: Running...", foreground="green")
            Thread(target=self.detect_objects_in_camera, daemon=True).start()

    def stop_detection(self):
        """Stop the detection."""
        self.is_running = False
        if self.video_capture:
            self.video_capture.release()
        self.status_label.config(text="Status: Stopped", foreground="orange")
        siren_sound.stop()

    def detect_objects_in_camera(self):
        """Detect objects using the camera and YOLOv4."""
        while self.is_running:
            ret, frame = self.video_capture.read()
            if not ret:
                self.status_label.config(text="Status: Camera Error", foreground="red")
                break

            frame = cv2.flip(frame, 1)

            blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
            net.setInput(blob)
            outputs = net.forward(net.getUnconnectedOutLayersNames())

            height, width = frame.shape[:2]
            boxes, confidences, class_ids = [], [], []
            weapon_detected = False

            for output in outputs:
                for detection in output:
                    scores = detection[5:]
                    class_id = np.argmax(scores)
                    confidence = scores[class_id]

                    if confidence > 0.5:
                        box = detection[:4] * np.array([width, height, width, height])
                        (center_x, center_y, w, h) = box.astype('int')
                        x = int(center_x - w / 2)
                        y = int(center_y - h / 2)

                        boxes.append([x, y, int(w), int(h)])
                        confidences.append(float(confidence))
                        class_ids.append(class_id)

                        if classes[class_id] in ["Handgun"]:
                            weapon_detected = True

            indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

            if len(indices) > 0:
                for i in indices.flatten():
                    x, y, w, h = boxes[i]
                    class_name = classes[class_ids[i]] if class_ids[i] < len(classes) else "Unknown"
                    label = f"{class_name}: {confidences[i]:.2f}"
                    color = (0, 255, 0)
                    cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                    cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            if weapon_detected:
                if not self.siren_playing:
                    siren_sound.play(-1) 
                    self.siren_playing = True

                current_time = time.time()
                if current_time - self.last_alert_time > 0.5:
                    send_alert("Alert! Handgun detected!")
                    self.last_alert_time = current_time
            # else:
            #     if self.siren_playing:
            #         siren_sound.stop()
            #         self.siren_playing = False

            cv2.imshow("Weapon Detection System", frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        self.stop_detection()
        cv2.destroyAllWindows()

    def quit_application(self):
        """Quit the application."""
        self.stop_detection()
        pygame.quit()
        self.root.destroy()

In [9]:
if __name__ == "__main__":
    root = tk.Tk()
    app = WeaponDetectionApp(root)
    root.mainloop()

Alert sent to user.
Alert sent to user.
Alert sent to user.
Alert sent to user.
Alert sent to user.
Alert sent to user.
Alert sent to user.
Alert sent to user.
